In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from tqdm import tqdm

POSTGRES_ADDRESS = 'db.panoply.io' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5439'
POSTGRES_USERNAME = 'vin.tang@gmail.com' ## CHANGE THIS TO YOURPANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'Cobalt27!' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
POSTGRES_DBNAME = 'playerslounge_prod' ## CHANGE THIS TO YOUR DATABASE NAME

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
password=POSTGRES_PASSWORD,
ipaddress=POSTGRES_ADDRESS,
port=POSTGRES_PORT,
dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

In [2]:
query = """
select contests.id, contests.__updatetime, contests.type, contests_plrsbefore.plr, contests.winner, contests.loser, contests_plrsbefore.userid, contests.status
from contests 
inner join contests_plrsbefore 
on contests.id = contests_plrsbefore.contests_id
where contests.console_game = 'ps4-madden21'
"""

In [3]:
df = pd.read_sql_query(query, cnx)
df.shape

(329532, 8)

In [4]:
df.head()

,id,__updatetime,type,plr,winner,loser,userid,status
0,7995092012829590,2020-08-21 19:15:34.342,H,61,4.439926e+14,7.038696e+15,7038695774196162,completed
1,5179645355963421,2020-08-21 20:10:45.923,H,54,7.893768e+15,6.613301e+15,7893767988794126,completed
2,5484846488566522,2020-08-21 20:10:45.923,H,67,NaN,NaN,8732438145053741,cancelled
3,2230477145682123,2020-08-21 21:13:48.940,H,84,NaN,NaN,8829685643037032,cancelled
4,3214806079020904,2020-08-21 22:19:37.525,H,79,NaN,NaN,8245774821481175,cancelled


In [6]:
lookup = {}

for _, row in tqdm(df.iterrows()):
    if row['status'] != "completed" or row['type'] != 'H':
        pass
    else:
        if row['id'] not in lookup.keys():
            if row['userid'] == row['winner']:
                lookup[row['id']] = {"winner": row['plr']}
            else:
                lookup[row['id']] = {"loser": row['plr']}
        else:
            if row['userid'] == row['winner']:
                lookup[row['id']]['winner'] = row['plr']
            else:
                lookup[row['id']]['loser'] = row['plr']
        lookup[row['id']]['time'] = row['__updatetime']

329532it [00:52, 6284.92it/s]


In [7]:
contest = []
winner = []
loser = []
time = []

for k, v in lookup.items():
    contest.append(k)
    winner.append(v['winner'])
    loser.append(v['loser'])
    time.append(v['time'])

In [9]:
outcomes = {'contest':contest, 'winner':winner, 'loser':loser, 'time':time}
data = pd.DataFrame.from_dict(outcomes)
data.head()

,contest,winner,loser,time
0,7995092012829590,64,61,2020-08-21 19:15:34.342
1,5179645355963421,54,59,2020-08-21 20:10:45.923
2,3186990526737690,64,59,2020-08-21 22:19:37.525
3,6369489012402744,70,62,2020-08-21 22:19:37.525
4,3091106470863898,71,62,2020-08-21 22:19:37.525


In [10]:
data.to_csv("ps4-madden21.csv")

In [15]:
correct = 0
count = 0

for k, v in lookup.items():
    if len(v.keys()) == 3:
        count += 1
        if v['winner'] >= v['loser']:
            correct += 1

In [16]:
correct/ count

0.6237570322393141

In [17]:
lookup['5148940047099294']

{'loser': 44, 'time': Timestamp('2020-08-21 19:15:34.342000'), 'winner': 74}